In [1]:
num_sample = 3000
num_burn = 200
sample_size = 800
n_cpu = 50

min_degree = 1
max_degree = 2

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 2


100%|██████████| 3200/3200 [00:16<00:00, 188.49it/s]


(0.6005634105430675,
 0.4139275,
 0.53743,
 0.34263422215430067,
 0.36713957041584466)

In [3]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
ground_truth1['psi_zero'] = res_truth1['psi_zero']

ground_truth1

100%|██████████| 200/200 [00:00<00:00, 279.26it/s]


{'average': 0.6596249999999999,
 'direct': 0.3172,
 'indirect': 0.01352500000000001,
 'spillover_effect': 0.023562500000000042,
 'psi_1_gamma': 0.7563,
 'psi_0_gamma': 0.4391,
 'psi_zero': 0.41553749999999995}

In [4]:
from utils import run_pll, delete_all_files_in_folder
from run_pll import run_dr_raw

treat_p = 0.7
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_raw')

Multiprocessing <function run_dr_raw at 0x7fa5cdabca40> in 1000 tasks, with 50 processes...
Multiprocessing finished.


In [5]:
from utils import run_pll
from run_pll import run_dr_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'psi_0_gamma_only': True,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_raw')

Multiprocessing <function run_dr_raw at 0x7fa5cdabca40> in 1000 tasks, with 50 processes...
Multiprocessing finished.


In [6]:
# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

100%|██████████| 1000/1000 [00:54<00:00, 18.50it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.93 , 0.947, 0.939, 0.928, 0.925, 0.944, 0.933]),
 'bias': array([-0.00364945, -0.00797781, -0.00090557, -0.00154059, -0.00762446,
         0.00035335,  0.00189394]),
 'mse': array([0.00758103, 0.01841978, 0.00857499, 0.01312787, 0.01415977,
        0.00495115, 0.01006666]),
 'var': array([0.00756771, 0.01835613, 0.00857417, 0.0131255 , 0.01410164,
        0.00495102, 0.01006308]),
 'ci_length': array([0.21752357, 0.37476304, 0.28344183, 0.31854668, 0.28200788,
        0.21101274, 0.24531051]),
 'true_effect': array([0.659625 , 0.3172   , 0.013525 , 0.0235625, 0.7563   , 0.4391   ,
        0.4155375])}

In [7]:
from tqdm import tqdm
from drnet import compute_avg_effects_std_from_raw

results1 = np.load(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)

var = []
var_sim = []
est = []
idx = 1
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(results1[i,:,idx],adj1,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(results1[i,:,idx]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

100%|██████████| 1000/1000 [00:08<00:00, 112.25it/s]

7.7973884391093735 7.842936670647254 8.050460494388577
